In [52]:
import pandas as pd
import utils
import numpy as np
import warnings
from tqdm import tqdm
from copairs.map import average_precision, mean_average_precision
import logging

logging.basicConfig(format="%(levelname)s:%(asctime)s:%(name)s:%(message)s")
logging.getLogger("copairs").setLevel(logging.INFO)

warnings.simplefilter(action="ignore", category=FutureWarning)

In [53]:
#operations = "wellpos_cc_var_mad_outlier_featselect_sphering_harmony_PCA_corrected"
batch_size = 20000
null_size = 20000
fdr = 0.1

In [3]:
# CRISPR controls - did not use this since the profies already has pert_type in it

#crispr_controls_df = pd.DataFrame(
#    {
#        "Metadata_JCP2022": ["JCP2022_805264", "JCP2022_800001", "JCP2022_800002"],
#        "Metadata_pert_type": ["poscon", "negcon", "negcon"],
##
#    index=[0, 1, 2],
#)

### Prepare the data

#### Filtering the CRISPR plates from the output file

In [54]:
raw_CRISPR_df = pd.read_parquet('C:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\CRISPRPipelineValidation\\profiles_wellpos_cc_var_mad_outlier_featselect_sphering_harmony_PCA_corrected.parquet')

In [55]:
raw_CRISPR_df.shape

(946431, 259)

In [70]:
#filtering 
crispr_df = raw_CRISPR_df[raw_CRISPR_df['Metadata_PlateType'] == 'CRISPR']

There are two columns of metadata for 'Metadata_Symbol' in the raw_orf_df - 'Metadata_Symbol_x' and 'Metadata_Symbol_y'. I have to check why there are two columns 

In [71]:
crispr_df.shape

(56821, 259)

In [72]:
crispr_df_cols = [c for c in crispr_df.columns if c.startswith("Metadata_")]

In [73]:
crispr_df_cols

['Metadata_Source',
 'Metadata_Plate',
 'Metadata_Well',
 'Metadata_JCP2022',
 'Metadata_broad_sample',
 'Metadata_Name',
 'Metadata_Vector',
 'Metadata_Transcript',
 'Metadata_Symbol_x',
 'Metadata_NCBI_Gene_ID_x',
 'Metadata_Taxon_ID',
 'Metadata_Gene_Description',
 'Metadata_Prot_Match',
 'Metadata_Insert_Length',
 'Metadata_pert_type',
 'Metadata_NCBI_Gene_ID_y',
 'Metadata_Symbol_y',
 'Metadata_Batch',
 'Metadata_PlateType',
 'Metadata_Row',
 'Metadata_Column',
 'Metadata_Microscope',
 'Metadata_Symbol',
 'Metadata_Approved_symbol',
 'Metadata_Locus',
 'Metadata_Chromosome',
 'Metadata_arm']

In [74]:
crispr_df['Metadata_NCBI_Gene_ID_x']

889610    NaN
889611    NaN
889612    NaN
889613    NaN
889614    NaN
         ... 
946426    NaN
946427    NaN
946428    NaN
946429    NaN
946430    NaN
Name: Metadata_NCBI_Gene_ID_x, Length: 56821, dtype: category
Categories (12602, object): ['1', '10', '100', '10000', ..., 'XLOC_l2_015578', 'XLOC_l2_015600', 'XLOC_l2_015937', 'eGFP']

In [75]:
crispr_df['Metadata_NCBI_Gene_ID_x'].isnull().all()

True

In [76]:
crispr_df['Metadata_Symbol'].isnull().all()

False

In [77]:
crispr_df['Metadata_Symbol_x'].isnull().all()


True

In [78]:
crispr_df['Metadata_Symbol_y'].isnull().all()


False

In [80]:
len(crispr_df['Metadata_Symbol_y'].notna())

56821

In [81]:
len(crispr_df['Metadata_Symbol'].notna())

56821

In [85]:
non_na_with_locs = crispr_df.where(crispr_df['Metadata_Symbol'].notna())

In [86]:
non_na_with_locs

,Metadata_Source,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_broad_sample,Metadata_Name,Metadata_Vector,Metadata_Transcript,Metadata_Symbol_x,Metadata_NCBI_Gene_ID_x,...,227,228,229,230,231,Metadata_Symbol,Metadata_Approved_symbol,Metadata_Locus,Metadata_Chromosome,Metadata_arm
889610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
889611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
889612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
889613,source_13,CP-CC9-R1-01,E06,JCP2022_806962,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.058746,0.159143,-0.055322,0.398936,-0.079594,TAZ,None,None,None,None
889614,source_13,CP-CC9-R1-01,F23,JCP2022_800002,NaN,NaN,NaN,NaN,NaN,NaN,...,0.039345,-0.157173,0.020388,0.011505,0.181091,non-targeting,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946426,source_13,CP-CC9-R8-02,F07,JCP2022_805270,NaN,NaN,NaN,NaN,NaN,NaN,...,0.023559,-0.057819,-0.016170,0.228918,-0.031281,PLOD2,PLOD2,3q24,3,3q
946427,source_13,CP-CC9-R8-02,F06,JCP2022_801824,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.202209,-0.004689,-0.157771,0.050958,-0.109742,DLAT,DLAT,11q23.1,11,11q
946428,source_13,CP-CC9-R8-02,F05,JCP2022_806749,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.053865,0.112224,0.374081,-0.089209,0.148852,SPR,SPR,2p13.2,2,2p
946429,source_13,CP-CC9-R8-02,F04,JCP2022_807630,NaN,NaN,NaN,NaN,NaN,NaN,...,0.051541,0.086372,0.050768,0.120248,-0.246816,VAV3,VAV3,1p13.3,1,1p


crispr_df['Metadata_Symbol_y'] and crispr_df['Metadata_Symbol'] is having around 5636 NaN which corresponds to 8 compound poscons and DMSO which can be filtered. Following are the JCP ID's of the 8 compound poscons and DMSO
['JCP2022_033924',
 'JCP2022_050797',
 'JCP2022_025848',
 'JCP2022_046054',
 'JCP2022_037716',
 'JCP2022_064022',
 'JCP2022_012818',
 'JCP2022_085227',
 'JCP2022_035095']

In [67]:
#Filtering/dropping the JCP ID's corresponding to 8 compound poscons and DMSO
crispr_df = crispr_df[~(crispr_df["Metadata_JCP2022"].isin(['JCP2022_033924',
 'JCP2022_050797',
 'JCP2022_025848',
 'JCP2022_046054',
 'JCP2022_037716',
 'JCP2022_064022',
 'JCP2022_012818',
 'JCP2022_085227',
 'JCP2022_035095']))]

In [69]:
len(crispr_df[(crispr_df["Metadata_JCP2022"].isin(['JCP2022_033924',
 'JCP2022_050797',
 'JCP2022_025848',
 'JCP2022_046054',
 'JCP2022_037716',
 'JCP2022_064022',
 'JCP2022_012818',
 'JCP2022_085227',
 'JCP2022_035095']))])

0

In [66]:
'JCP2022_033924' in crispr_df['Metadata_Symbol_y'].values

False

In [63]:
crispr_df.shape

(56821, 259)

In [43]:
differences = crispr_df[crispr_df['Metadata_Symbol_y'] != crispr_df['Metadata_Symbol']]
print(differences)

       Metadata_Source Metadata_Plate Metadata_Well Metadata_JCP2022  \
889610       source_13   CP-CC9-R1-01           F02   JCP2022_033924   
889611       source_13   CP-CC9-R1-01           F01   JCP2022_064022   
889612       source_13   CP-CC9-R1-01           E24   JCP2022_085227   
889615       source_13   CP-CC9-R1-01           E23   JCP2022_033924   
889616       source_13   CP-CC9-R1-01           F24   JCP2022_037716   
...                ...            ...           ...              ...   
903550       source_13   CP-CC9-R8-02           E24   JCP2022_085227   
903551       source_13   CP-CC9-R8-02           F01   JCP2022_064022   
903552       source_13   CP-CC9-R8-02           F02   JCP2022_033924   
903553       source_13   CP-CC9-R8-02           H24   JCP2022_046054   
903554       source_13   CP-CC9-R8-02           P24   JCP2022_046054   

       Metadata_broad_sample Metadata_Name Metadata_Vector  \
889610                   NaN           NaN             NaN   
889611     

In [48]:
only_two = differences[['Metadata_Symbol_y', 'Metadata_Symbol']]

In [51]:
only_two.isnull().all()

Metadata_Symbol_y    True
Metadata_Symbol      True
dtype: bool

In [15]:
#dropping the columns ['Metadata_Symbol_x'] and ['Metadata_NCBI_Gene_ID_x'] since they have NaNs
crispr_df = crispr_df.drop(['Metadata_Symbol_x', 'Metadata_NCBI_Gene_ID_x', 'Metadata_Symbol'], axis=1)

KeyError: "['Metadata_Symbol_x', 'Metadata_NCBI_Gene_ID_x', 'Metadata_Symbol'] not found in axis"

In [13]:
#renaming the ['Metadata_Symbol_y'] and ['Metadata_NCBI_Gene_ID_y'] to be used downstream 
crispr_df = crispr_df.rename(columns={'Metadata_NCBI_Gene_ID_y':'Metadata_NCBI_Gene_ID', 'Metadata_Symbol_y':'Metadata_Symbol'})

In [14]:
crispr_df.shape

(56821, 256)

In [15]:
crispr_df.to_parquet('C:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\CRISPRPipelineValidation\\profiles_crispr.parquet')

The size of the original file is (51185, 263)

#### Add annotations - not running this since the profiles alreayd has the metadata in it 

In [11]:
#    pd.read_csv(
#        "../00.download-and-process-annotations/output/crispr_metadata.tsv.gz", sep="\t"
#    )
#   .merge(crispr_controls_df, on="Metadata_JCP2022", how="left")
#    .fillna(value={"Metadata_pert_type": "trt"})
#)
#compound_metadata_df = pd.read_csv(
#    "../datasets/metadata/compound.csv.gz", usecols=["Metadata_JCP2022"]
#).assign(
#    Metadata_pert_type=lambda x: np.where(
#        x["Metadata_JCP2022"] == "JCP2022_999999", "empty", "poscon"
#    )
#)

#metadata_df = pd.concat(
#    [crispr_metdata_df, compound_metadata_df],
#    join="outer",
#    ignore_index=True,
#)

#crispr_df = crispr_df.merge(metadata_df, on="Metadata_JCP2022", how="inner")
#crispr_df.shape

(56821, 651)

#### Remove `poscon` wells.

In [16]:
crispr_df = crispr_df.query('Metadata_pert_type!="poscon"').reset_index(drop=True)
crispr_df.shape

(51744, 256)

#### Remove featues with `nan` values.
These need to be removed as the `nan` values will cause the mean average precision calculation to fail.

In [17]:
crispr_df = utils.remove_nan_features(crispr_df)
crispr_df.shape

Removed nan features: []


(51744, 256)

#### Remove reagents with only one replicate

In [18]:
crispr_df.Metadata_JCP2022.value_counts()[
        crispr_df.Metadata_JCP2022.value_counts() == 1
    ].reset_index()['Metadata_JCP2022'].tolist()

['JCP2022_807435', 'JCP2022_807044', 'JCP2022_803433']

In [19]:
#changed the code to the column name
reagents_with_one_replicate = (
    crispr_df.Metadata_JCP2022.value_counts()[
        crispr_df.Metadata_JCP2022.value_counts() == 1
    ]
    .reset_index()["Metadata_JCP2022"] #.reset_index()["index"]
    .tolist()
)


In [20]:

crispr_df = crispr_df[~crispr_df.Metadata_JCP2022.isin(reagents_with_one_replicate)]
crispr_df.shape

(51741, 256)

### Calculate mAP for each ORF perturbation

In [21]:
# Adding a new column for negative control
crispr_df["Metadata_negcon"] = np.where(crispr_df["Metadata_pert_type"] == "negcon", 1, 0)

In [22]:
pos_sameby = ["Metadata_JCP2022"]
pos_diffby = []
neg_sameby = ["Metadata_Plate"]
neg_diffby = ["Metadata_negcon"]

In [23]:
metadata_df = utils.get_metadata(crispr_df)
feature_df = utils.get_featuredata(crispr_df)
feature_values = feature_df.values

In [24]:
result = average_precision(
    metadata_df, feature_values, pos_sameby, pos_diffby, neg_sameby, neg_diffby, batch_size=batch_size
)

INFO:2024-08-22 17:41:40,142:copairs:Indexing metadata...
INFO:2024-08-22 17:41:40,218:copairs:Finding positive pairs...
INFO:2024-08-22 17:41:49,079:copairs:Finding negative pairs...
INFO:2024-08-22 17:41:50,214:copairs:Computing positive similarities...
INFO:2024-08-22 17:42:07,637:copairs:Computing negative similarities...
INFO:2024-08-22 17:42:09,462:copairs:Building rank lists...
INFO:2024-08-22 17:42:33,607:copairs:Computing average precision...
INFO:2024-08-22 17:42:34,329:copairs:Creating result DataFrame...
INFO:2024-08-22 17:42:34,336:copairs:Finished.


In [25]:
# Remove negcon
result = result.query('Metadata_pert_type!="negcon"').reset_index(drop=True)

In [26]:
agg_result = (
    mean_average_precision(result, pos_sameby, null_size=null_size, threshold=fdr, seed=12527)
    .rename(columns={'average_precision': 'mean_average_precision'})
)

INFO:2024-08-22 17:42:44,775:copairs:Computing null_dist...


INFO:2024-08-22 17:42:44,864:copairs:Computing p-values...


In [27]:
agg_result.to_csv(f"C:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\CRISPRPipelineValidation\\PhenotypicActivity_CRISPR.csv.gz", index=False)